# Operator and State Classes

In [203]:
import numpy
import math
#This class defines operators
class Operator:
    action = -1
    
    def __init__(self, act):
        self.action = act

#This class defines state 
class State:
    # This variable stores the tile configuration
    tiledArr = numpy.empty((3,3),dtype='int')
    
    # These two variables store the current location of empty tile
    freeTileI = 0
    freeTileJ = 0
    
    # This variable is to retrieve the path after finding a solution
    previous = None
    
    # This stores the cost from start node
    gVal = 0
    
    #Initializing the state with a given a tile configuration (arr)
    def __init__(self, arr):
        self.tiledArr = numpy.empty((3,3),dtype='int')
        for i in range(3):
            for j in range(3):
                self.tiledArr[i][j] = arr[i][j]
                
                #Empty tile is represented as a -1 in the 2D array
                if(self.tiledArr[i][j] == -1):
                    self.freeTileI = i
                    self.freeTileJ = j
    
    #This function returns a 1 if two tile configurations are the same and 0 otherwise
    def isEqual(self, s):
        for i in range(3):
            for j in range(3):
                if(self.tiledArr[i][j] != s.tiledArr[i][j]):
                    return 0
        return 1
    
    #This function applies an operator O on the current state (self) configuration 
    #and returns a new state configuration s1
    def applyOperator(self, o):
        s1 = State(self.tiledArr)
        s1.previous = self
        s1.gVal = self.gVal + 1
        #0 - West, 1 - South, 2 - East and 3 - North'
        if(o.action == 0):
            if(self.freeTileJ - 1 >= 0):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI][self.freeTileJ-1]
                s1.freeTileJ -= 1
        elif(o.action == 1):
            if(self.freeTileI + 1 < 3):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI+1][self.freeTileJ]
                s1.freeTileI += 1
        elif(o.action == 2):
            if(self.freeTileJ + 1 < 3):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI][self.freeTileJ+1]
                s1.freeTileJ += 1
        elif(o.action == 3):
            if(self.freeTileI - 1 >= 0):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI-1][self.freeTileJ]
                s1.freeTileI -= 1
        
        s1.tiledArr[s1.freeTileI][s1.freeTileJ] = -1
                
        return s1
    
    #This function prints the state configuration
    def printState(self):
        for i in range(3):
            for j in range(3):
                print("%d"%self.tiledArr[i][j],end=' ')
            print()
        print("*******")

#  Utility Functions

In [204]:
#Function that indicates whether a state is present in a list
def isPresentStateInList(state, searchList):
    for elem in searchList:
        if(state.isEqual(elem) == 1):
            return 1

    return 0

#Function that indicates whether a state is present in a priority list
def isPresentStateInPriorityList(state, searchList):
    for [elem, dist] in searchList:
        if(state.isEqual(elem) == 1):
            return 1
        
    return 0

#Number of tiles in wrong position
def calculateHeuristicValue(current, goal):
    counter = 0
    for i in range(3):
        for j in range(3):
            if(current.tiledArr[i][j] != goal.tiledArr[i][j]):
                counter+=1

    return counter

# Sum of manhattan distance of each tile from its goal position
def calculateHeuristicValueManhattan(current, goal):
    xposcurrent = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    yposcurrent = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    xposgoal =  [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    yposgoal =  [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    for i in range(3):
        for j in range(3):
            if current.tiledArr[i][j]!=-1:
                xposcurrent[current.tiledArr[i][j]] = i
                yposcurrent[current.tiledArr[i][j]] = j
            if goal.tiledArr[i][j]!=-1:
                xposgoal[goal.tiledArr[i][j]] = i
                yposgoal[goal.tiledArr[i][j]] = j
    totdist=0
    for i in range(1,9):
        totdist+= (math.fabs(xposcurrent[i]-xposgoal[i])+math.fabs(yposcurrent[i]-yposgoal[i]))
    
    return totdist

def insertStateInPriorityQueue(searchList, state, distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(distanceToGoal < searchList[i][1]):
            index = i
            break
    
    if(len(searchList) == 0 or index == -1):
        searchList.append([state,distanceToGoal])
    else:
        searchList.insert(index, [state,distanceToGoal])

#Reinserts the element in priority queue if the new value is less than the 
#value present in queue.
def checkAndUpdateStateInPriorityQueue(searchList,state,distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(state.isEqual(searchList[i][0])):
            if distanceToGoal < searchList[i][1]:
                    index = i
            break
            
    if index!=-1:
        searchList.remove([searchList[index][0],searchList[index][1]])
        insertStateInPriorityQueue(searchList, state, distanceToGoal)
        

def retrievePathFromState(state):
    visitedstatelist = []
    visitedstatelist.append(state)
    #state.printState()
    prev = state.previous
    counter = 0
    while(prev != None):
        visitedstatelist.append(prev)
        prev = prev.previous
        counter +=1
    visitedstatelist.reverse()
    for i in range(len(visitedstatelist)):
        visitedstatelist[i].printState()
    print("Size of path is ",counter)

# Depth First Search

In [205]:
def searchDFS(start, goal):
    openList = []
    closedList = []
    counter=0
    #Add start state to openList
    openList.append(start)
    
    #flag to exit search
    flag = 1
    
    while(len(openList) > 0):       
        counter+=1
        #Retrieve the first state, eliminate it from open list and add it to closed list
        current = openList[0]
        openList.remove(current)
        closedList.append(current)
        #Obtain successor states and add them to open list if they are not in closed list and open list
        for i in range(4):
            o = Operator(i)
            nextState = current.applyOperator(o)
            #If we have reached goal, retrieve path from start to goal
            if nextState.isEqual(goal):
                retrievePathFromState(nextState)
                flag = 0
                break
            
            if(isPresentStateInList(nextState, closedList) == 0 and isPresentStateInList(nextState, openList) == 0):
                openList.insert(0, nextState)

        if(flag == 0 or counter > 5000):
            print("Number of nodes explored ",counter)
            break
            


# Breadth First Search

In [206]:
def searchBFS(start, goal):
    openList = []
    closedList = []
    
    #Add start state to openList
    openList.append(start)
    flag = 1
    counter=0
    while(len(openList) > 0):
        counter+=1
        #Retrieve the first state, eliminate it from open list and add it to closed list
        current = openList[0]
        openList.remove(current)
        closedList.append(current)
        #Obtain successor states and add them to open list if they are not in closed list and open list
        for i in range(4):
            o = Operator(i)
            nextState = current.applyOperator(o)
             #If we have reached goal, retrieve path from start to goal
            if nextState.isEqual(goal):
                retrievePathFromState(nextState)
                flag = 0
                break
            #If not in closed and open lists, then add to open list
            if(isPresentStateInList(nextState, closedList) == 0 and isPresentStateInList(nextState, openList) == 0):
                openList.append(nextState)
                
        if(flag == 0 or counter > 5000):
            print("Number of nodes explored ",counter)
            break
            
       

# Best First Search 

In [207]:
def searchBestFirst(start, goal):

    openList = []
    closedList = []
    openList.append([start,calculateHeuristicValue(start,goal)])
    
    flag = 1
    counter = 0
    while(len(openList) > 0):
        counter+=1
        #Retrieve the first state, eliminate it from open list and add it to closed list
        current = openList[0]
        openList.remove(current)
        closedList.append(current[0])
       
        #Obtain successor states and add them to open list if they are not in closed list and open list
        for i in range(4):
            o = Operator(i)
            nextState = current[0].applyOperator(o)
            #If we have reached goal, retrieve path from start to goal
            if nextState.isEqual(goal):
                retrievePathFromState(nextState)
                flag = 0
                break
            if(isPresentStateInList(nextState, closedList) == 0 and isPresentStateInPriorityList(nextState, openList) == 0):
                insertStateInPriorityQueue(openList, nextState, calculateHeuristicValue(nextState, goal))
            #If present in openlist, check if the value needs to be updated. If yes, update the value and reinsert into queue
            #at correcct position
            elif isPresentStateInPriorityList(nextState, openList):
                checkAndUpdateStateInPriorityQueue(openList,nextState,calculateHeuristicValue(nextState, goal))
        if(flag == 0):
            print("Number of nodes explored ",counter)
            break

# A* Search

In [208]:
def searchAStar(start, goal):
    openList = []
    closedList = []
    openList.append([start,calculateHeuristicValue(start,goal)]) #start.gval is 0 
    flag = 1
    counter = 0
    while(len(openList) > 0):
        counter+=1        
      
        #Retrieve the first state, eliminate it from open list and add it to closed list
        current = openList[0]
        openList.remove(current)
        closedList.append(current[0])
        
                   
        #Obtain successor states and add them to open list if they are not in closed list and open list
        for i in range(4):
            o = Operator(i)
            nextState = current[0].applyOperator(o)
            #If we have reached goal, retrieve path from start to goal
            if nextState.isEqual(goal):
                retrievePathFromState(nextState)
                flag = 0
                break
            if(isPresentStateInList(nextState, closedList) == 0 and isPresentStateInPriorityList(nextState, openList) == 0):
                insertStateInPriorityQueue(openList, nextState, nextState.gVal + calculateHeuristicValue(nextState, goal))
          
             #If present in openlist, check if the value needs to be updated. If yes, update the value and reinsert into queue
            #at the correcct position
            #gval is already updated to the new value after applying the operator.
            elif isPresentStateInPriorityList(nextState, openList):
                checkAndUpdateStateInPriorityQueue(openList,nextState,nextState.gVal + calculateHeuristicValue(nextState, goal))
        if(flag == 0):
            print("Number of nodes explored ",counter)
            break
       

# Try Search Functions

In [209]:
# Informed searches, search for lesser or equal nodes to uninformed searches.

#Best first searches for less nodes as compared to A*. Solution obtained is same
#BFS takes more than 3000 nodes. 
#DFS could not find a solution after searching for 5000 nodes
start = State([[4,5,6],[-1,3,8],[7,1,2]])
goal = State([[6,8,2],[5,3,1],[4,-1,7]])

# Best first and A* same number of nodes explored. Solution obtained is same
#start = State([[2,8,3],[1,6,4],[7,-1,5]])
#goal = State([[1,2,3],[8,-1,4],[7,6,5]])

#Best first explores less nodes but provides non optimal solution.  A* explores around 6000 nodes before finding a solution.
#start = State([[5,-1,8],[4,2,1],[7,3,6]])
#goal = State([[1,2,3],[4,5,6],[7,8,-1]])

#DFS can not find a path after searching 5000 nodes. BFS finds a path after searching 2 nodes
#start = State([[4,5,6],[3,8,-1],[7,1,2]])
#goal = State([[4,5,6],[-1,3,8],[7,1,2]])

#DFS - explores lot more nodes and provides a non optimal path
#start = State([[-1,5,6],[4,3,8],[7,1,2]])
#goal = State([[4,5,6],[7,3,8],[-1,1,2]])

#DFS explores less nodes than BFS, obtains same solution
#start = State([[4,5,6],[7,3,8],[1,2,-1]])
#goal = State([[4,-1,5],[7,3,6],[1,2,8]])


start.printState()

print("Running A*")

searchAStar(start, goal)
print("Finished A*")

print("Running Best First")
searchBestFirst(start, goal)
print("Finished Best First")

print("Running Breadth First")
searchBFS(start, goal)
print("Finished Breadth First")

print("Running Depth First")
searchDFS(start, goal)
print("Finished Depth First")

4 5 6 
-1 3 8 
7 1 2 
*******
Running A*
4 5 6 
-1 3 8 
7 1 2 
*******
-1 5 6 
4 3 8 
7 1 2 
*******
5 -1 6 
4 3 8 
7 1 2 
*******
5 6 -1 
4 3 8 
7 1 2 
*******
5 6 8 
4 3 -1 
7 1 2 
*******
5 6 8 
4 3 2 
7 1 -1 
*******
5 6 8 
4 3 2 
7 -1 1 
*******
5 6 8 
4 3 2 
-1 7 1 
*******
5 6 8 
-1 3 2 
4 7 1 
*******
-1 6 8 
5 3 2 
4 7 1 
*******
6 -1 8 
5 3 2 
4 7 1 
*******
6 8 -1 
5 3 2 
4 7 1 
*******
6 8 2 
5 3 -1 
4 7 1 
*******
6 8 2 
5 3 1 
4 7 -1 
*******
6 8 2 
5 3 1 
4 -1 7 
*******
Size of path is  14
Number of nodes explored  157
Finished A*
Running Best First
4 5 6 
-1 3 8 
7 1 2 
*******
-1 5 6 
4 3 8 
7 1 2 
*******
5 -1 6 
4 3 8 
7 1 2 
*******
5 6 -1 
4 3 8 
7 1 2 
*******
5 6 8 
4 3 -1 
7 1 2 
*******
5 6 8 
4 3 2 
7 1 -1 
*******
5 6 8 
4 3 2 
7 -1 1 
*******
5 6 8 
4 3 2 
-1 7 1 
*******
5 6 8 
-1 3 2 
4 7 1 
*******
-1 6 8 
5 3 2 
4 7 1 
*******
6 -1 8 
5 3 2 
4 7 1 
*******
6 8 -1 
5 3 2 
4 7 1 
*******
6 8 2 
5 3 -1 
4 7 1 
*******
6 8 2 
5 3 1 
4 7 -1 
*******
6 8 2 
5